In [1]:
!pip install better-profanity

In [2]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import col
from better_profanity import profanity
from operator import add

# spark_session = SparkSession\
#         .builder\
#         .master("spark://sparkmaster:7077") \
#         .appName("test")\
#         .config("spark.dynamicAllocation.enabled", True)\
#         .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
#         .config("spark.shuffle.service.enabled", False)\
#         .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
#         .config("spark.executor.cores",2)\
#         .config("spark.driver.port",9999)\
#         .config("spark.blockManager.port",10005)\
#         .getOrCreate()
# spark_context = spark_session.sparkContext

spark = SparkSession.builder \
    .appName("MongoDB Integration") \
    .master("spark://sparkmaster:7077") \
    .config("spark.mongodb.read.connection.uri", "mongodb://mongo_db:27017/reddit.reddit_data?readPreference=primaryPreferred") \
    .config("spark.mongodb.input.uri", "mongodb://mongo_db:27017/reddit.reddit_data") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.0') \
    .config("spark.dynamicAllocation.enabled", True)\
    .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
    .config("spark.shuffle.service.enabled", False)\
    .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
    .getOrCreate()

# Load data from MongoDB
df = spark.read.format("mongo").load()

In [3]:
profanity.load_censor_words()

In [10]:
from pyspark.sql import functions, Column

df2 = df.select(col("subreddit"), col("normalizedBody"))
def profanity_udf(text):
    cleaned = profanity.censor(text)
    return cleaned.count("****") 

prof_udf = functions.udf(profanity_udf)

df3 = df2.withColumn('pgRated', prof_udf(col('normalizedBody')))


subreddit_count = df3.groupBy("subreddit").agg({"pgRated": "sum"}).withColumnRenamed("sum(pgRated)", "total_censored")
final_count = subreddit_count.orderBy(subreddit_count.total_censored.desc())



In [ ]:
final_count.show(10)